In [1]:
# Import Block
import numpy as np
from rdkit import Chem
from openbabel import pybel
from openff.toolkit.topology import Molecule,Topology
from openff.toolkit.utils import RDKitToolkitWrapper
import openmm
from simtk.openmm import app
from openmm import unit
from openmm.app import PDBFile
from openff.toolkit.typing.engines.smirnoff import ForceField
from openmm.vec3 import Vec3
import warnings
warnings.simplefilter("ignore")


In [2]:
# Wrapper and FF setup
# Use RDKit wrapper
rdktkw = RDKitToolkitWrapper()

# Loading setup parameters
forcefield = ForceField('openff-2.0.0.offxml')

In [3]:
# load pdb with one copy of pdb file
off_mol = Molecule.from_pdb_and_smiles('7101899.pdb', "CC1=CN=C(C(=C1OC)C)C[S@@](=O)C2=NC3=C(N2)C=C(C=C3)OC")
# load supercell pdb file (2x2x2) into topology
pdb_file = PDBFile('7101899_supercell.pdb')
off_top = Topology.from_openmm(pdb_file.topology, [off_mol])
# Create MD simulation inputs
system = forcefield.create_openmm_system(off_top)
integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
platform = openmm.Platform.getPlatformByName('Reference')

/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [4]:
# create simulation
simulation = openmm.app.Simulation(pdb_file.topology, system, integrator, platform)
# set initial positions from pdbfile
positions = pdb_file.getPositions()
simulation.context.setPositions(positions)

In [5]:
# set reporters
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', 1)
state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    1,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [6]:
simulation.context.setPositions(positions)
simulation.saveState('initial')
orig_potential = simulation.context.getState(getEnergy=True).getPotentialEnergy()
#state_data_reporter.report(simulation, simulation.context.getState())
print('Initial Energy ' + str(orig_potential))
print('Minimizing Energy!')
simulation.minimizeEnergy()
min_state = simulation.context.getState(getEnergy=True, getPositions=True, getForces=True)
min_potential = min_state.getPotentialEnergy()
print('Final Energy = ' + str(min_potential))

Initial Energy 1313.8142291462432 kJ/mol
Minimizing Energy!
Final Energy = -158.13780344456245 kJ/mol


In [ ]:
# Report if system uses periodic boundary conditions and forces used
print(simulation.system.usesPeriodicBoundaryConditions())
print(simulation.system.getForces())

In [22]:
# Need block to feed parameters to minimizer, where forces provide 3*n derivatives of energy
# wrt position, +3 more derivatives of energy wrt box vectors
# This block is a work in progress
forces = simulation.context.getState(getForces=True).getForces()
p_box_pos = simulation.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions()

x0 = np.array([2,0,2,0,0,2]) # a_x, b_x, b_y, c_x, c_y, c_z
def box_energy(sim,x,positions):
    # x will be np array of all inputs
    sim.system.setDefaultPeriodicBoxVectors(Vec3(x[0],0,0),Vec3(x[1],x[2],0),Vec3(x[3],x[4],x[5]))
    print(sim.system.getForces())
    new_integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
    new_sim = openmm.app.Simulation(pdb_file.topology, system, new_integrator, platform)
    new_sim.context.setPositions(positions)
    return new_sim.context.getState(getEnergy=True).getPotentialEnergy()
energy = box_energy(simulation,x0,positions)
print(energy)
#print(forces)



[<openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7f6d0fb527e0> >, <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f6d0f38eb10> >, <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f6d0f38ea50> >, <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f6d0f38e8a0> >]
1313.8142291462432 kJ/mol


In [7]:
simulation.step(1)


In [8]:
import pymol
pymol.cmd.load('7101899_supercell.pdb','initial')
pymol.cmd.load('trajectory.pdb','final')
rmsd = pymol.cmd.align('final','initial',cycles=0)
print(rmsd[0])


1.9751797914505005
